# Syncing course directory files to Dropbox

In [ ]:
# Helper functions to convert full paths to relative paths
# and vice-versa


def full_path(root, rel_path):
    from pathlib import Path
    return f'{(Path(root)/rel_path).as_posix()}'


def rel_path(root, full_path):
    from pathlib import Path
    return f'{Path(full_path).relative_to(root).as_posix()}'

In [ ]:
# Wrapper class with methods for managing data from csv


class CSV:
    @staticmethod
    def read(filename):
        '''Read in (previous) local_entries from CSV.'''
        try:
            f = open(filename, 'r')
        except FileNotFoundError:
            return {}
        except Exception:
            raise
        else:
            local_entries = {}
            for row in f:
                # Use tuple assignment to extract path as key, rev as value
                key, rev, mtime = row.strip().split(':::')
                # convert NoneType properly
                rev = None if rev == 'None' else rev
                mtime = float(mtime)
                local_entries[key] = {'rev': rev, 'mtime': mtime}
            f.close()
            print(f'local_entries read from {filename}')
        return local_entries

    @staticmethod
    def export(filename, entries):
        '''Store (current) local_entries to CSV.'''
        try:
            f = open(filename, 'w')
        except Exception:
            raise
        else:
            count = 0
            for path, metadata in entries.items():
                f.write(':::'.join([path, str(metadata['rev']),
                                    str(metadata['mtime'])])+'\n')
                count += 1
            f.close()
            print(f'local_entries saved to {filename}')
        return count

In [ ]:
# This code sets up the Dropbox API class so it can be used later.
import os
from dropbox_api import RateLimitError, APIError, DBApi

# Use python-dotenv to load environment variables for better security
from dotenv import load_dotenv
load_dotenv()

ADMIN = os.environ.get("admin")
TOKEN = os.environ.get("dbtoken")

# Go to the correct directory for importing API
os.chdir(ADMIN)

# Instantiate the Dropbox API object
db = DBApi(TOKEN)

In [ ]:
# Get list of files (relative path) from local folders
import os


ignore = ['.ipynb_checkpoints',
          '.ipython',
          '.bash_history',
          '.bashrc',
          '.local',
          '.npm',
          '.conda',
          '.cache',
          '.jupyter',
          '.fontconfig',
          '.config',
          '.profile',
          '.bash_logout',
          '.git',
          '__pycache__',
          ]
local_root = Path(os.environ.get("HOME")).as_posix()
os.chdir(local_root)
print(f'Changing directory to {local_root}')
rel_entries = [rel_path(local_root, file) for file in
               Path(local_root).rglob('*')]

# Get previous local_entries and update it
local_entries = CSV.read(f'{ADMIN}/local_entries.csv')  # Returns empty dict if file does not exist
for rpath in rel_entries:
    if all([pattern not in rpath for pattern in ignore]):
        local_path = full_path(local_root, rpath)
        if local_path.startswith('..'):
            import pdb; pdb.set_trace()
        try:
            mtime = local_entries[local_path]['mtime']
        except KeyError:
            mtime = Path(local_path).lstat().st_mtime
            local_entries[local_path] = {'rev': None, 'mtime': mtime}
            print(f'LC_ADD   : {local_path}')
        except Exception as e:
            raise
        else:
            curr_mtime = Path(local_path).lstat().st_mtime
            if curr_mtime == mtime:
                pass
            else:
                local_entries[local_path]['rev'] = None
                local_entries[local_path]['mtime'] = curr_mtime
                print(f'LC_RESET : {local_path}')

del rel_entries

In [ ]:
from pathlib import Path

# Get list of Dropbox entries as db_entries using Dropbox API
# db_entries is a dict with db_path as key and entry as value
db_entries = {entry['path_display']: entry for entry in
              db.list_folder('')['entries']}

In [ ]:
# Delete Dropbox entries that are not in local_entries

# Let's keep a list of entries to delete from db_entries
# We can use this list to update db_entries later instead of
# polling the API again

to_delete = []
for db_path in db_entries.keys():
    relpath = rel_path(db.root, db_path)
    local_path = full_path(local_root, relpath)
    if local_path not in local_entries and local_path != db.root:
        try:
            db.delete(db_path)
        except Exception as e:
            raise
        else:
            to_delete.append(db_path)
            print(f'DB_DELETE: {db_path}')

for db_path in to_delete:
    del db_entries[db_path]
del to_delete

In [ ]:
# Create folder in Dropbox if:
# 1) Not in db_entries
# Upload file to Dropbox if:
# 2) not in local_entries (have rev = None),
# 3) have rev not matching latest Dropbox rev
for localpath, metadata in local_entries.items():
    if localpath.startswith('..'):
        import pdb; pdb.set_trace()
    relpath = rel_path(local_root, localpath)
    db_path = full_path(db.root, relpath)
    rev = metadata['rev']
    # (1)
    if Path(localpath).is_dir() and db_path not in db_entries.keys():
        try:
            response = db.create_folder(db_path)
        except APIError as e:
            raise
            # TODO: error logging
        except Exception:
            raise
        else:
            if db_path == response['metadata']['path_display']:
                print(f'DB_CREATE: {localpath}')
            else:
                print(f'DB_FAIL  : CREATE {localpath}')
    # (2)
    if Path(localpath).is_file() and (db_path not in db_entries.keys()
                                      # (3)
                                      or rev is None
                                      or db_entries[db_path]['rev'] != rev):
        try:
            response = db.upload_file(localpath,
                                      db_path=db_path,
                                      root=local_root,
                                      )
        except RateLimitError as e:
            raise
            # TODO: set wait timer if 409 encountered
        except APIError as e:
            raise
            # TODO: Error logging
        except Exception as e:
            raise
        else:
            # store updated rev in local_entries
            local_entries[localpath]['rev'] = response['rev']
            print(f'DB_UPLOAD: {db_path}')

In [ ]:
CSV.export(f'{ADMIN}/local_entries.csv', local_entries)